# Creating dataset to test evaluation

during the creation of the evaluation method it was found that the method was inable to deal with output of the model that does not adhere to the expected output structure. A dataset of model output that can slightly defiate from the the expected ouput is needed to test methodes on how to deal with output that doesn't adhere to the expected structre

In [11]:
from transformers import (
    T5Tokenizer,
    AutoTokenizer,
    T5ForConditionalGeneration,
    Seq2SeqTrainingArguments,
    Seq2SeqTrainer,
    DataCollatorForSeq2Seq,
    GenerationConfig
)
import torch
from datasets import load_dataset, concatenate_datasets
from wasabi import msg
import random
import pandas as pd
from tqdm import tqdm
from pathlib import Path
from os.path import abspath

## Setting home directory

In [4]:
home_dir = Path(abspath("")).parent

msg.info(f"Home directory: {home_dir}")

# Change when creating a new dataset for test purposes
resulting_dataset_path = home_dir.joinpath(
    "data/testing_eval/cdr_seq2rel/test_trained_T5-3b.csv"
)

ℹ Home directory: /home/lgrootde/Generative-re-tests


In [5]:
# Load the config
import yaml
with open(home_dir.joinpath('config/config_T5-3b_cdr.yaml')) as f:
    config = yaml.load(f, Loader=yaml.FullLoader)

### Load dataset

In [12]:
dataset = load_dataset(
        config['dataset_vars']['type'], 
        data_dir=home_dir.joinpath(config['dataset_vars']['dir']),
        column_names=config['dataset_vars']['column_names']
        )

dataset_eval = concatenate_datasets([dataset[split].select(range(1,501)) for split in config['splits_for_validation']]) # remove first row that contains column names

### Load model

In [7]:
# Load model and tokenizer
model_name = config['model_name']
device_map = {"": 0}

tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True, legacy=False)

# Load model after training
model = T5ForConditionalGeneration.from_pretrained(
    home_dir.joinpath("results/checkpoint-100"),
    device_map=device_map,
    local_files_only=True
) # we specificly use T5 for Conditional generations because it has a language modeling head

/home/lgrootde/Generative-re-tests/venv/lib/python3.12/site-packages/transformers/models/t5/tokenization_t5_fast.py:171: FutureWarning: This tokenizer was incorrectly instantiated with a model max length of 512 which will be corrected in Transformers v5.
For now, this behavior is kept to avoid breaking backwards compatibility when padding/encoding with `truncation is True`.
- Be aware that you SHOULD NOT rely on google-t5/t5-3b automatically truncating your input to 512 when padding/encoding.
- If you want to encode/pad to sequences longer than 512 you can either instantiate this tokenizer with `model_max_length` or pass `max_length` when encoding/padding.
- To avoid this warning, please instantiate this tokenizer with `model_max_length` set to your preferred value.
  warnings.warn(
Loading checkpoint shards: 100%|██████████| 3/3 [10:47<00:00, 215.89s/it]


### Specify generation config

In [9]:
# generation_config = GenerationConfig(
#     decoder_start_token_id=0,
#     eos_token_id=1,
#     pad_token_id=0,
#     max_length=512,
#     num_beam=4
# )

### Run model over dataset

In [13]:
test_eval_method_dataset = {
    "input" : [],
    "expected output" : [],
    "predicted" : []
}
for row in tqdm(test_dataset):
    input = row["input"]
    input_ids = tokenizer(input, return_tensors="pt").input_ids.to('cuda')
    output = model.generate(input_ids, max_length=256)
    decoded_output = tokenizer.decode(output[0], skip_special_tokens=True)

    test_eval_method_dataset["input"].append(input)
    test_eval_method_dataset["expected output"].append(row["relations"])
    test_eval_method_dataset["predicted"].append(decoded_output)

100%|██████████| 500/500 [08:58<00:00,  1.08s/it]


### Add output to dataset and save new dataset

In [14]:
dataset = pd.DataFrame.from_dict(test_eval_method_dataset)
dataset.to_csv(resulting_dataset_path, index=False)